In [44]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [45]:
import pandas as pd
from scipy import stats
import numpy as np
from sklearn import preprocessing
from datetime import datetime
from matplotlib import pyplot

In [46]:
df = pd.read_csv("./data/no1_train.csv")
df["y_prev"] = df["y"].shift(1)
df.describe()

,hydro,micro,thermal,wind,river,total,y,sys_reg,flow,y_prev
count,225088.000000,225088.000000,225088.000000,225088.000000,225088.0,225088.000000,225088.000000,225088.000000,225088.000000,225087.000000
mean,1888.334127,224.189826,21.803497,55.703266,0.0,2190.031963,8.907921,-8.284305,-1999.055081,8.907633
std,401.000305,58.559855,3.579229,47.053033,0.0,444.077730,320.262539,43.748315,1311.682286,320.263221
min,683.438000,80.371000,0.000000,0.000000,0.0,849.732000,-1579.680903,-828.000000,-5541.200000,-1579.680903
25%,1625.029000,185.017000,22.100000,15.704000,0.0,1916.523000,-142.226700,0.000000,-2996.700000,-142.226960
50%,1933.916000,225.938000,22.100000,41.848000,0.0,2248.874000,-11.282351,0.000000,-1953.700000,-11.283806
75%,2175.595000,269.558000,24.100000,88.723000,0.0,2497.792000,127.454943,0.000000,-906.900000,127.455918
max,2995.524000,349.271000,25.700000,176.000000,0.0,3351.974000,2956.333317,474.000000,723.400000,2956.333317


In [47]:
preprocess_columns = ["hydro", "micro", "thermal", "wind", "total", "sys_reg", "flow", "y_prev", "y"]

# Data preprocessing

### Clamp then normalize

In [48]:
def filter_column_based_on_quantile(df, q, col):
    q_low = df[col].quantile(q)
    q_hi  = df[col].quantile(1-q)
    df = df[(df[col] < q_hi) & (df[col] > q_low)]
    return df

for col in preprocess_columns:
    df = filter_column_based_on_quantile(df, 0.001, col)
    #df[col] = (df[col] - df[col].min()) / (df[col].max() - df[col].min())

df.describe()

,hydro,micro,thermal,wind,river,total,y,sys_reg,flow,y_prev
count,218476.000000,218476.000000,218476.000000,218476.000000,218476.0,218476.000000,218476.000000,218476.000000,218476.000000,218476.000000
mean,1886.566494,224.103826,21.820337,55.677761,0.0,2188.169702,2.947032,-7.862550,-1997.643444,2.991219
std,391.938330,57.853667,3.430808,46.851155,0.0,434.098625,293.817362,38.170766,1310.400513,294.217412
min,714.533000,80.674000,4.500000,0.002000,0.0,903.810000,-887.243293,-512.000000,-5234.180000,-954.853235
25%,1626.809000,185.014000,22.100000,15.892000,0.0,1918.362000,-141.735665,0.000000,-2998.660000,-141.742713
50%,1930.184000,225.557000,22.100000,42.024000,0.0,2244.954000,-12.575697,0.000000,-1946.000000,-12.587640
75%,2168.250000,268.708000,24.100000,88.319000,0.0,2490.977000,123.901878,0.000000,-905.400000,123.909845
max,2901.979000,346.216000,25.600000,169.024000,0.0,3242.057000,2038.149064,120.000000,526.850000,2216.103712


In [49]:
df

,start_time,hydro,micro,thermal,wind,river,total,y,sys_reg,flow,y_prev
1,2019-01-09 14:15:00,1591.839,141.928,11.5,74.428,0.0,1819.695,330.114330,10.0,-4417.4,341.330021
2,2019-01-09 14:20:00,1591.839,141.928,11.5,74.428,0.0,1819.695,323.877221,10.0,-4417.4,330.114330
3,2019-01-09 14:25:00,1591.839,141.928,11.5,74.428,0.0,1819.695,296.438463,10.0,-4417.4,323.877221
4,2019-01-09 14:30:00,1591.839,141.928,11.5,74.428,0.0,1819.695,269.097132,10.0,-4417.4,296.438463
5,2019-01-09 14:35:00,1591.839,141.928,11.5,74.428,0.0,1819.695,232.113498,10.0,-4417.4,269.097132
...,...,...,...,...,...,...,...,...,...,...,...
225083,2021-03-01 03:05:00,1942.486,191.881,20.1,106.431,0.0,2266.098,141.618915,0.0,-2252.9,141.026229
225084,2021-03-01 03:10:00,1942.486,191.881,20.1,106.431,0.0,2266.098,129.043820,0.0,-2252.9,141.618915
225085,2021-03-01 03:15:00,1942.486,191.881,20.1,106.431,0.0,2266.098,94.907548,0.0,-2252.9,129.043820
225086,2021-03-01 03:20:00,1942.486,191.881,20.1,106.431,0.0,2266.098,102.589040,0.0,-2252.9,94.907548


### Add time features

In [50]:
df['start_time'] = pd.to_datetime(df['start_time'], format="%Y-%m-%d %H:%M:%S")

In [51]:
df["time_of_day"] = df["start_time"].dt.hour
df["time_of_week"] = df["start_time"].dt.dayofweek
df["time_of_year"] = df["start_time"].dt.month-1
#df = pd.get_dummies(df, columns=['time_of_day', "time_of_week", "time_of_year"])

In [53]:
df.describe()

,hydro,micro,thermal,wind,river,total,y,sys_reg,flow,y_prev,time_of_day,time_of_week,time_of_year
count,218476.000000,218476.000000,218476.000000,218476.000000,218476.0,218476.000000,218476.000000,218476.000000,218476.000000,218476.000000,218476.000000,218476.000000,218476.000000
mean,1886.566494,224.103826,21.820337,55.677761,0.0,2188.169702,2.947032,-7.862550,-1997.643444,2.991219,11.540512,2.989834,5.233746
std,391.938330,57.853667,3.430808,46.851155,0.0,434.098625,293.817362,38.170766,1310.400513,294.217412,6.923604,1.991830,3.548513
min,714.533000,80.674000,4.500000,0.002000,0.0,903.810000,-887.243293,-512.000000,-5234.180000,-954.853235,0.000000,0.000000,0.000000
25%,1626.809000,185.014000,22.100000,15.892000,0.0,1918.362000,-141.735665,0.000000,-2998.660000,-141.742713,6.000000,1.000000,2.000000
50%,1930.184000,225.557000,22.100000,42.024000,0.0,2244.954000,-12.575697,0.000000,-1946.000000,-12.587640,12.000000,3.000000,5.000000
75%,2168.250000,268.708000,24.100000,88.319000,0.0,2490.977000,123.901878,0.000000,-905.400000,123.909845,18.000000,5.000000,8.000000
max,2901.979000,346.216000,25.600000,169.024000,0.0,3242.057000,2038.149064,120.000000,526.850000,2216.103712,23.000000,6.000000,11.000000


In [54]:
#columns_to_use = ["time_of_day", "time_of_week", "time_of_year", "hydro", "micro", "thermal", "wind", "total", "sys_reg", "flow", "y_prev", "y"]
columns_to_drop = ["start_time", "river"]
df = df.drop(columns=columns_to_drop)
df.to_csv("./data/preprocessed_data.csv", index=False)
df

,hydro,micro,thermal,wind,total,y,sys_reg,flow,y_prev,time_of_day,time_of_week,time_of_year
1,1591.839,141.928,11.5,74.428,1819.695,330.114330,10.0,-4417.4,341.330021,14,2,0
2,1591.839,141.928,11.5,74.428,1819.695,323.877221,10.0,-4417.4,330.114330,14,2,0
3,1591.839,141.928,11.5,74.428,1819.695,296.438463,10.0,-4417.4,323.877221,14,2,0
4,1591.839,141.928,11.5,74.428,1819.695,269.097132,10.0,-4417.4,296.438463,14,2,0
5,1591.839,141.928,11.5,74.428,1819.695,232.113498,10.0,-4417.4,269.097132,14,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...
225083,1942.486,191.881,20.1,106.431,2266.098,141.618915,0.0,-2252.9,141.026229,3,0,2
225084,1942.486,191.881,20.1,106.431,2266.098,129.043820,0.0,-2252.9,141.618915,3,0,2
225085,1942.486,191.881,20.1,106.431,2266.098,94.907548,0.0,-2252.9,129.043820,3,0,2
225086,1942.486,191.881,20.1,106.431,2266.098,102.589040,0.0,-2252.9,94.907548,3,0,2


In [55]:
df_val = pd.read_csv("./data/no1_validation.csv")

df_val["y_prev"] = df_val["y"].shift(1)
df_val['start_time'] = pd.to_datetime(df_val['start_time'], format="%Y-%m-%d %H:%M:%S")
df_val["time_of_day"] = df_val["start_time"].dt.hour
df_val["time_of_week"] = df_val["start_time"].dt.dayofweek
df_val["time_of_year"] = df_val["start_time"].dt.month-1
#df_val = pd.get_dummies(df_val, columns=['time_of_day', "time_of_week", "time_of_year"])
df_val = df_val.drop(columns=columns_to_drop)
df_val.to_csv("./data/preprocessed_validation_data.csv", index=False)
df_val

,hydro,micro,thermal,wind,total,y,sys_reg,flow,y_prev,time_of_day,time_of_week,time_of_year
0,1942.490,191.881,20.1,106.431,2266.102,52.119595,0.0,-2252.90,NaN,3,0,2
1,1942.490,191.881,20.1,106.431,2266.102,90.374541,0.0,-2252.90,52.119595,3,0,2
2,1942.490,191.881,20.1,106.431,2266.102,88.435063,0.0,-2252.90,90.374541,3,0,2
3,1942.490,191.881,20.1,106.431,2266.102,53.375518,0.0,-2252.90,88.435063,3,0,2
4,1942.490,191.881,20.1,106.431,2266.102,65.496683,0.0,-2252.90,53.375518,3,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...
28131,2416.610,316.111,20.1,25.427,2778.248,-51.999568,-110.0,-135.00,-41.175585,19,6,5
28132,2416.610,316.111,20.1,25.427,2778.248,-55.399064,-110.0,-135.00,-51.999568,19,6,5
28133,2416.610,316.111,20.1,25.427,2778.248,-31.270359,-110.0,-138.42,-55.399064,19,6,5
28134,2350.079,316.121,20.1,32.935,2719.235,-54.007893,-110.0,-146.97,-31.270359,20,6,5


,start_time,hydro,micro,thermal,wind,river,total,y,sys_reg,flow
0,2021-03-01 03:30:00,1942.490,191.881,20.1,106.431,0.0,2266.102,52.119595,0.0,-2252.90
1,2021-03-01 03:35:00,1942.490,191.881,20.1,106.431,0.0,2266.102,90.374541,0.0,-2252.90
2,2021-03-01 03:40:00,1942.490,191.881,20.1,106.431,0.0,2266.102,88.435063,0.0,-2252.90
3,2021-03-01 03:45:00,1942.490,191.881,20.1,106.431,0.0,2266.102,53.375518,0.0,-2252.90
4,2021-03-01 03:50:00,1942.490,191.881,20.1,106.431,0.0,2266.102,65.496683,0.0,-2252.90
...,...,...,...,...,...,...,...,...,...,...
28131,2021-06-06 19:45:00,2416.610,316.111,20.1,25.427,0.0,2778.248,-51.999568,-110.0,-135.00
28132,2021-06-06 19:50:00,2416.610,316.111,20.1,25.427,0.0,2778.248,-55.399064,-110.0,-135.00
28133,2021-06-06 19:55:00,2416.610,316.111,20.1,25.427,0.0,2778.248,-31.270359,-110.0,-138.42
28134,2021-06-06 20:00:00,2350.079,316.121,20.1,32.935,0.0,2719.235,-54.007893,-110.0,-146.97
